In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from sklearn.mixture import GaussianMixture
import PySimpleGUI as sg
import os
from PIL import Image

warnings.filterwarnings('ignore')
data = []
original_data=data

In [2]:
dist_list = ["cauchy", "chi2","pearson3",
             "expon", "exponpow","dweibull",
             "lognorm", "powerlaw", "uniform",
             "beta",  "gamma","norm", "rayleigh"]


In [3]:
#available distributions
# 'alpha','anglit','arcsine','beta','betaprime','bradford','burr',
# 'burr12','cauchy','chi','chi2','cosine','dgamma','dweibull','erlang',
# 'expon','exponnorm','exponweib','exponpow','f','fatiguelife','fisk',
# 'foldcauchy','foldnorm','frechet_r','frechet_l','genlogistic','genpareto',
# 'gennorm','genexpon','genextreme','gausshyper','gamma','gengamma','genhalflogistic',
# 'gilbrat','gompertz','gumbel_r','gumbel_l','halfcauchy','halflogistic','halfnorm',
# 'halfgennorm','hypsecant','invgamma','invgauss','invweibull','johnsonsb','johnsonsu',
# 'kstwobign','laplace','levy','levy_l','logistic','loggamma','loglaplace','lognorm','lomax',
# 'maxwell','mielke','nakagami','ncx2','ncf','nct','norm','pareto','pearson3','powerlaw','powerlognorm',
# 'powernorm','rdist','reciprocal','rayleigh','rice','recipinvgauss','semicircular','t','triang','truncexpon',
# 'truncnorm','tukeylambda','uniform','vonmises','vonmises_line','wald','weibull_min','weibull_max'

In [4]:
#outlier_removal

#goes threw johnson transformation
def jt(data):
    yeo = getattr(stats, 'yeojohnson')
    data, lmbda2 = yeo(data)
    return data

#removes outliers with 3 S.D away (3 z-score)
def nd(data):
    for y in data:
        sd=np.std(data, ddof=1)
        mean=np.mean(data)
        ucl= mean +3*sd
        lcl=mean -3*sd
        data1 = data[data<ucl]
        data_new =data1[data1>lcl]
        x=len(data)-len(data_new)
        data=data_new
        return data
        if x == 0:
            break
    
#turns johnson transformed data back to normal   
def jt_filtered(data):
    data_maxfit =jt(data)<max(nd(jt(data)))
    data_minfit =jt(data)>min(nd(jt(data)))
    data =data*data_maxfit*data_minfit
    return data

In [5]:
# distribution_fitter
def dist_fit(data):
    best_stat=np.inf
    best_dist=[]

    #finding best distribution
    for dist in dist_list:
        x = getattr(stats,dist)
        params = x.fit(data)
        stat,p = stats.kstest(data, dist, args=params)
        if best_stat>stat:
            best_stat = stat
            best_dist = dist

    #plotting best fit and data and showing outliers
    x = getattr(stats,best_dist)
    params = x.fit(data)
    x_fit = np.linspace(min(data),max(data), len(data))
    pdf = x.pdf(x_fit, *params,)
    if len(data)<501:
        nbins=len(data)
    else:
        nbins = 500
    fig = px.histogram(x= original_data,histnorm = 'probability density', title=(str(best_dist)+",stat="+str(best_stat)),nbins=nbins)
    fig.add_scatter(x =x_fit, y =pdf)
    fig.add_vline(max(data))
    fig.add_vline(min(data))
    if not os.path.exists("output_graphs"):
        os.mkdir("output_graphs")
    fig.write_image("output_graphs/fig1.png")
    

In [6]:
#qqplot
def qqplot(data):
    #grid setup
    n_cols = int(np.ceil(np.sqrt(len(dist_list))))
    n_rows = int(np.ceil(len(dist_list)/n_cols))
    fig, ax  = plt.subplots(n_cols, n_rows, figsize=(n_cols*3, n_rows*3))
    axs = ax.flatten()
    i = 0
    y=len(axs)
    for x in axs:
        if y == len(dist_list):
            break
        y=y-1
        axs[y].set_visible(False)

    #qq plots
    for t in dist_list:
        x = getattr(stats,t)
        params = x.fit(data)
        sm.qqplot(data,x,fit=params,line="45", ax = axs[i])
        axs[i].set_title(t)
        i+=1
    plt.tight_layout()
    fig.patch.set_facecolor('white')
    if not os.path.exists("output_graphs"):
        os.mkdir("output_graphs")
    fig.savefig("output_graphs/fig1.png")
    plt.close(fig)
    
    img = Image.open("output_graphs/fig1.png")
    img = img.resize((512, 512), Image.ANTIALIAS)
    img.save("output_graphs/fig1.png")

In [7]:
#finding optimal ammount of distributions
def BIC(data):
    reshaped_data= np.array(data).reshape(-1,1)
    min_bic=-1
    for x in range(1,11):
        GMdata=GaussianMixture(n_components=x,covariance_type='full')
        GMdata.fit(reshaped_data)
        bic=GMdata.bic(reshaped_data)
        if bic < min_bic or min_bic==-1:
            min_bic = bic
            n_optimal=x
    return (n_optimal,reshaped_data)

In [8]:
#graphing the gaussian mixture
def GaussianMix(data):
    
    #collecting the gaussian mixture data
    GMdata=GaussianMixture(n_components=n_optimal,covariance_type='full')
    GMdata.fit(BIC(data)[1])

    #plotting
    x_axis = np.arange(min(original_data),max(original_data))
    y_avg=np.array(0,)
    if len(data)<501:
        nbins=len(data)
    else:
        nbins = 500
    fig = px.histogram(x= original_data,histnorm = 'probability density',title=("gaussian Mixture of "+str(n_optimal)+" components"),nbins=nbins)
    fig.add_vline(max(data))
    fig.add_vline(min(data))

    x_axis = np.arange(min(data),max(data))
    for n in range(n_optimal):
        y_axis = stats.norm.pdf(x_axis, float(GMdata.means_[n]), np.sqrt(float(GMdata.covariances_[n])))*GMdata.weights_[n]
        fig.add_scatter(x = x_axis, y = y_axis,line=dict(color="black",width=3),line_shape='spline')
        y_avg = y_avg+y_axis
    
    fig.add_scatter(x =x_axis, y =y_avg,line=dict(color="darkred",width=2),line_shape='spline')
    if not os.path.exists("output_graphs"):
        os.mkdir("output_graphs")
    fig.write_image("output_graphs/fig1.png")

In [ ]:
#The User Interface

Icon = "..."
sg.theme('Dark Grey5')
option = (1)

#layout
left_column = [    
    [sg.Text('input Data (csv or excel)')],
    [sg.Input(enable_events = True,key="file"),sg.FileBrowse(file_types = (("xls files","*.xls"),("csv files","*.csv"),("xlsx files","*.xlsx")))],
    [sg.Text("Select column")],
    [sg.Combo([],enable_events=True,size=(43,0),key = "column")],
    [sg.Text('Anaylsis Type')],
    [sg.Radio("Distribution Fitter","connect_radio",enable_events=True,key = "fit"),sg.Radio("QQ-plots","connect_radio",enable_events=True,key = "qq-plot"),sg.Radio("Gaussian Mixture","connect_radio",enable_events=True,key ="mix")],
    [sg.Button("Enter",key = "enter"), sg.Button("Exit",key = "exit")]
                ]
right_column = [
    [sg.Text("---output---", justification='center')],
    [sg.Image(key="graph", filename='')]
    ]

layout = [[sg.Column(left_column), sg.VSeperator(),sg.Column(right_column)]]

#Collecting input
window = sg.Window('Data Analysis Tool', layout,icon=Icon)
while True:
    event, values = window.read()
    if event == "exit":
        break
    if event == "file":
        file = values["file"]
        try:
            df_data =pd.read_excel(file)
        except:
            try:
                df_data =pd.read_csv(file, encoding = 'unicode_escape')
            except:
                print("not a suitable file type")
        window["column"].update(value = '', values=list(df_data.columns))
            
    if event == "fit":
        option = (1)
        
    if event == "mix":
        option = (2)
    
    if event == "qq-plot":
        option = (3)
        
    if event == "enter":
        data = (df_data[values["column"]].dropna(how="all"))
        original_data=data
        data = jt_filtered(data)
        
        if option == (1):
            dist_fit(jt_filtered(data))
            window["graph"].update(filename="output_graphs/fig1.png")
            
        if option == (3):
            qqplot(data)
            window["graph"].update(filename="output_graphs/fig1.png")
            
        if option == (2):
            n_optimal = BIC(data)[0]
            GaussianMix(data)
            window["graph"].update(filename="output_graphs/fig1.png")

window.close()